In [36]:
#listName = ['ceiling', 'floor', 'wall','beam','column','window','door','table','chair','sofa','bookcase','board','clutter']

In [37]:
import numpy as np
import torch
import GeometricTools
from sklearn.cluster import DBSCAN

In [38]:
ceiling = torch.load('s3dis_example/celling.pt')
ceiling_ins = torch.load('s3dis_example/ceiling_ins.pt')
print(ceiling.shape)

floor = torch.load('s3dis_example/floor.pt')
floor_ins = torch.load('s3dis_example/floor_ins.pt')
print(floor.shape)

wall = torch.load('s3dis_example/wall.pt')
wall_ins = torch.load('s3dis_example/wall_ins.pt')
print(wall.shape)

column = torch.load('s3dis_example/column.pt')
column_ins = torch.load('s3dis_example/column_ins.pt')
print(column.shape)

door = torch.load('s3dis_example/door.pt')
door_ins = torch.load('s3dis_example/door_ins.pt')
print(door.shape)

window = torch.load('s3dis_example/window.pt')
window_ins = torch.load('s3dis_example/window_ins.pt')
print(window.shape)

board = torch.load('s3dis_example/board.pt')
board_ins = torch.load('s3dis_example/board_ins.pt')
print(board.shape)

clutter = torch.load('s3dis_example/clutter.pt')
clutter_ins = torch.load('s3dis_example/clutter_ins.pt')
print(clutter.shape)

room_with_wall = torch.load('s3dis_example/room_with_wall.pt')
room_not_wall = torch.load('s3dis_example/room_not_wall.pt')
room_not_cc = torch.load('s3dis_example/room_not_cc.pt')


table = torch.load('s3dis_example/table.pt')
table_ins = torch.load('s3dis_example/table_ins.pt')
print(table.shape)

chair = torch.load('s3dis_example/chair.pt')
chair_ins = torch.load('s3dis_example/chair_ins.pt')
print(chair.shape)


bookcase = torch.load('s3dis_example/bookcase.pt')
bookcase_ins = torch.load('s3dis_example/bookcase_ins.pt')
print(bookcase.shape)

sofa = torch.load('s3dis_example/sofa.pt')
sofa_ins = torch.load('s3dis_example/sofa_ins.pt')
print(sofa.shape)



(8192, 3)
(8192, 3)
(32768, 3)
(32768, 3)
(16384, 3)
(16384, 3)
(8192, 3)
(385024, 3)
(73728, 3)
(65536, 3)
(81920, 3)
(8192, 3)


In [39]:

def column_gen(range_x, range_y, height, fix_point):
    thick           = 0.03
    point_thick     = 1
    fix_point       = fix_point

    # Follow x

    # range_x         = max_x 
    # range_y         = max_y
    range_z         = height

    percent_2piece  = (range_x*range_z) / ((range_y*range_z) +  (range_x*range_z))

    percent_xz      = range_x / range_z
    percent_yz      = range_y / range_z

    num_point_p1    = fix_point * percent_2piece 
    num_point_p2    = fix_point - num_point_p1

    num_point_y1    = point_thick
    num_point_x1    = int(np.sqrt( num_point_p1/ num_point_y1 * percent_xz )) 
    num_point_z1    = int(num_point_p1 / num_point_x1 / num_point_y1) + 1

    num_point_x2    = point_thick
    num_point_y2    = int(np.sqrt( num_point_p2/ num_point_x2 * percent_yz )) 
    num_point_z2    = int(num_point_p2 / num_point_x2 / num_point_y2) + 1

    interval_x1     = np.linspace(0, range_x, num_point_x1) 
    interval_y1     = np.linspace(0, thick, num_point_y1) 
    interval_z1     = np.linspace(0, range_z, num_point_z1) 

    interval_x2     = np.linspace(0, thick, num_point_x2) 
    interval_y2     = np.linspace(0, range_y , num_point_y2) 
    interval_z2     = np.linspace(0, range_z, num_point_z2) 

    piece1          = np.array([[a,b,c] for a in interval_x1 for b in interval_z1 for c in interval_y1])[:int(num_point_p1) + 1]
    piece2          = np.array([[a,b,c] for a in interval_x2 for b in interval_z2 for c in interval_y2])[:int(num_point_p2) + 1]
    
    return piece1, piece2, piece1, piece2





def column_completer(column_ins, column, max_x_room, max_y_room, max_z_room):
    #column = column[:, [0, 2, 1]]
    new_column = []
    new_column_ins = []
    for i in np.unique(column_ins):
        mask = i == column_ins
        column_i = column[mask]
        instance = np.ones((column_i.shape[0]))*i
        height =  max_z_room
        min_x, max_x, min_y, max_y = np.min(column_i[:,0]), np.max(column_i[:,0]), np.min(column_i[:,2]), np.max(column_i[:,2])
        #print(min_x, max_x, min_y, max_y)
        range_x, range_y = (max_x-min_x), (max_y - min_y)
        piece1, piece2, piece3, piece4 = column_gen(range_x, range_y, height, 10000)
        if max_x <= max_x_room*3/4 and max_y <=max_y_room*1/4 : # first
            print("1")
            if min_x <= 0.5 and min_y <= 0.5: # case corner
                piece1[:,2]     += range_y
                piece2[:,0]     += range_x
                column_i         = np.concatenate((piece1, piece2), axis = 0)
            else: # case middle
                another_pice = np.copy(piece4)
                piece1[:,2]     += range_y
                piece2[:,0]     += range_x
                column_i         = np.concatenate((another_pice, piece1, piece2), axis = 0)
                column_i[:,0] += min_x
        elif max_x >=3/4*max_x_room and max_y <=3/4 * max_y_room:
            print("2")
            if max_x >= max_x_room - 0.5 and min_y <= 0.5: # case corner
                piece1[:,2]     += range_y
                column_i         = np.concatenate((piece1, piece2), axis = 0)
                column_i[:,0]    += (max_x_room - range_x)
            else:
                another_pice = np.copy(piece3)
                piece1[:,2]     += range_y
                column_i         = np.concatenate((another_pice, piece1, piece2), axis = 0)
                column_i[:,0]    += (max_x_room - range_x)
                column_i[:,2]    += min_y
        elif max_x <= 1/4 *max_x_room and max_y >=1/4 * max_y_room:
            print("3")
            if max_y >= max_y_room - 0.5 and min_x <= 0.5:
                piece2[:,0]     += range_x
                column_i         = np.concatenate((piece1, piece2), axis = 0)
                column_i[:,2]    += (max_y_room - range_y)
            else:
                another_pice = np.copy(piece3)
                another_pice[:,2] += range_y
                piece2[:,0]     += range_x
                column_i         = np.concatenate((another_pice, piece1, piece2), axis = 0)
                column_i[:,2]    += min_y

        elif max_x >=1/4*max_x_room and max_y >=3/4*max_y_room:
            print("4")
            if max_x >= max_x_room - 0.5 and max_y >= max_y_room - 0.5:
                column_i         = np.concatenate((piece1, piece2), axis = 0)
                column_i[:,0]    += (max_x_room - range_x)
                column_i[:,2]    += (max_y_room - range_y)
            else:
                another_pice = np.copy(piece4)
                another_pice[:,0] += range_x
                column_i         = np.concatenate((another_pice, piece1, piece2), axis = 0)
                column_i[:,0]    += min_x
                column_i[:,2]    += (max_y_room - range_y)
        new_column.append(column_i)
        new_column_ins.append(instance)
        #GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(column_i), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
    new_column = np.concatenate(new_column, axis =0)
    new_column_ins = np.array(new_column_ins)
    #new_column = new_column[:, [0, 2, 1]]
    return new_column, new_column_ins

print(column)
column = column[:, [0, 2, 1]]
print(column)


print(np.max(column[:,0]), np.max(column[:,2]))
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(column), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
new_column, new_column_ins = column_completer(column_ins, column, np.max(column[:,0]), np.max(column[:,2]), np.max(column[:,1]))


new_column = new_column[:, [0, 2, 1]]



GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(new_column), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])






[[6.066 0.517 2.243]
 [5.215 0.518 0.493]
 [5.214 0.046 3.057]
 ...
 [5.268 6.143 2.308]
 [5.607 5.844 2.476]
 [5.254 5.912 2.908]]
[[6.066 2.243 0.517]
 [5.215 0.493 0.518]
 [5.214 3.057 0.046]
 ...
 [5.268 2.308 6.143]
 [5.607 2.476 5.844]
 [5.254 2.908 5.912]]
6.099 6.42
2
3
3
4


In [40]:
# for i in np.unique(table_ins):
#     mask = table_ins == i
#     table_i = table[mask]
#     np.save(f'C:/data/ShapeNet55/more/table_{i}.npy', table_i)
    

# for i in np.unique(chair_ins):
#     mask = chair_ins == i
#     chair_i = chair[mask]
#     np.save(f'C:/data/ShapeNet55/more/chair_{i}.npy', chair_i)
    
# for i in np.unique(bookcase_ins):
#     mask = bookcase_ins == i
#     bookcase_i = bookcase[mask]
#     np.save(f'C:/data/ShapeNet55/more/bookcase_{i}.npy', bookcase_i)

In [41]:
# np.save('C:/data/ShapeNet55/more/sofa.npy', sofa)
# np.save('C:/data/ShapeNet55/more/table.npy', sofa)
# np.save('C:/data/ShapeNet55/more/chair.npy', sofa)
# np.save('C:/data/ShapeNet55/more/bookcase.npy', sofa)

In [42]:
def wall_x(min_x, max_x, min_y, max_y, min_z, max_z, fix_point):
    
    range_x         = max_x - min_x
    range_y         = max_y - min_y
    range_z         = max_z - min_z

    percent_xz       = range_x / range_z

    num_point_y     = 1
    num_point_x     = int(np.sqrt( fix_point/ num_point_y * percent_xz ))  + 1
    num_point_z     = int(fix_point / num_point_x / num_point_y) 

    # Đây là code để tạo tường random lộn xộn điểm

    # interval_x1 = np.random.uniform(min_x, max_x, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_y1 = np.random.uniform(min_y, thick, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_z1 = np.random.uniform(min_z, max_z, size = (num_point_x, num_point_z, num_point_y) ) 

    #wall_x1 = np.stack((interval_x1, interval_z1, interval_y1), axis = 3).reshape(-1,3)[:fix_point]

    interval_x1 = np.linspace(min_x, max_x, num_point_x) 
    interval_y1 = np.linspace(min_y, max_y, num_point_y) 
    interval_z1 = np.linspace(min_z, max_z, num_point_z) 

    

    wall = np.array([[a,b,c] for a in interval_x1 for b in interval_y1 for c in interval_z1])[:fix_point]
    #GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(wa), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
    return wall




In [43]:
def wall_y(min_x, max_x, min_y, max_y, min_z, max_z, fix_point):

    range_x         = max_x - min_x
    range_y         = max_y - min_y
    range_z         = max_z - min_z

    percent_yz       = range_y / range_z

    num_point_x     = 1
    num_point_y     = int(np.sqrt( fix_point/ num_point_x * percent_yz )) +1
    num_point_z     = int(fix_point / num_point_y / num_point_x) 

    # interval_x1 = np.random.uniform(min_x, thick, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_y1 = np.random.uniform(min_y, max_y, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_z1 = np.random.uniform(min_z, max_z, size = (num_point_x, num_point_z, num_point_y) ) 
    # wall_y1 = np.stack((interval_x1, interval_z1, interval_y1), axis = 3).reshape(-1,3)[:fix_point]

    interval_x1 = np.linspace(min_x, max_x, num_point_x) 
    interval_y1 = np.linspace(min_y, max_y, num_point_y) 
    interval_z1 = np.linspace(min_z, max_z, num_point_z) 
    


    wall = np.array([[a,b,c] for a in interval_x1 for b in interval_y1 for c in interval_z1])[:fix_point]

    return wall

In [44]:
def floor1(min_x, max_x, min_y, max_y, min_z, max_z, fix_point):
    range_x         = max_x - min_x
    range_y         = max_y - min_y
    range_z         = max_z - min_z

    percent_xy       = range_x / range_y

    num_point_z     = 1
    num_point_x     = int(np.sqrt( fix_point/ num_point_z * percent_xy )) 
    num_point_y     = int(fix_point / num_point_x / num_point_z) + 1

    # interval_x1 = np.random.uniform(min_x, max_x, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_y1 = np.random.uniform(min_y, max_y, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_z1 = np.random.uniform(min_z, thick, size = (num_point_x, num_point_z, num_point_y) ) 
    # floor1 = np.stack((interval_x1, interval_z1, interval_y1), axis = 3).reshape(-1,3)[:fix_point]

    interval_x1 = np.linspace(min_x, max_x, num_point_x) 
    interval_y1 = np.linspace(min_y, max_y, num_point_y) 
    interval_z1 = np.linspace(min_z, max_z, num_point_z) 

    floor = np.array([[a,b,c] for a in interval_x1 for b in interval_y1 for c in interval_z1])[:fix_point]
    GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(floor), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
    return floor

In [45]:
def ceiling1(min_x, max_x, min_y, max_y, min_z, max_z, fix_point):
    range_x         = max_x - min_x
    range_y         = max_y - min_y
    range_z         = max_z - min_z

    percent_xy       = range_x / range_y

    num_point_z     = 1
    num_point_x     = int(np.sqrt( fix_point/ num_point_z * percent_xy )) 
    num_point_y     = int(fix_point / num_point_x / num_point_z) + 1

    # interval_x1 = np.random.uniform(min_x, max_x, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_y1 = np.random.uniform(min_y, max_y, size = (num_point_x, num_point_z, num_point_y) ) 
    # interval_z1 = np.random.uniform(min_z, thick, size = (num_point_x, num_point_z, num_point_y) ) 
    # floor1 = np.stack((interval_x1, interval_z1, interval_y1), axis = 3).reshape(-1,3)[:fix_point]

    interval_x1 = np.linspace(min_x, max_x, num_point_x) 
    interval_y1 = np.linspace(min_y, max_y, num_point_y) 
    interval_z1 = np.linspace(min_z, max_z, num_point_z) 

    ceiling = np.array([[a,b,c] for a in interval_x1 for b in interval_y1 for c in interval_z1])[:fix_point]
    GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(ceiling), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
    return ceiling

In [18]:
def duc_tuong1(object_ins, objecter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.04):
    mask_door = objecter[:,1] <= (max_y + min_y) / 2
    door_true = object_ins[mask_door]
    for j in np.unique(door_true):# take out each door
        this_door = objecter[object_ins == j]
        this_masked_door = this_door[:,1] <= (max_y + min_y) / 2 + 0.05 # take out the point near the wall
        still_this_door = this_door[this_masked_door]
        
        if still_this_door.shape == (0,3):
            return wall_i
        # take min, max x,z to cut the wall
        min_x, max_x, min_z, max_z = np.min(still_this_door[:,0]), np.max(still_this_door[:,0]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
        hup1 = np.logical_and(wall_i[:,0] <= max_x, wall_i[:,0] >= min_x)
        hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
        hup_het = np.logical_and(hup1, hup2)
        
        temp_wall_i = wall_i[hup_het]
        cut_wall_i = wall_i[~hup_het]
        # 
        for point in still_this_door:
            distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,0], temp_wall_i[:,2]])) - np.array([point[0], point[2]]), axis=1)
            mask_distance = distances <= threshold # only take out distance that <= 0.0001
            temp_wall_i = temp_wall_i[~mask_distance]
        
        wall_i = np.concatenate((cut_wall_i, temp_wall_i))
    return wall_i

In [19]:
def duc_tuong2(object_ins, objecter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.04):
    mask_door = objecter[:,1] >= (max_y + min_y) / 2
    door_true = object_ins[mask_door]
    for j in np.unique(door_true):# take out each door
        this_door = objecter[object_ins== j]
        print("this_door", this_door)
        this_masked_door = this_door[:,1] >= (max_y + min_y) / 2 + 0.05 # take out the point near the wall
        still_this_door = this_door[this_masked_door]
        print("still_this_door", still_this_door.shape)
        if still_this_door.shape == (0,3):
            return wall_i
        # take min, max x,z to cut the wall
        min_x, max_x, min_z, max_z = np.min(still_this_door[:,0]), np.max(still_this_door[:,0]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
        hup1 = np.logical_and(wall_i[:,0] <= max_x, wall_i[:,0] >= min_x)
        hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
        hup_het = np.logical_and(hup1, hup2)
        
        temp_wall_i = wall_i[hup_het]
        cut_wall_i = wall_i[~hup_het]
        # 
        for point in still_this_door:
            distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,0], temp_wall_i[:,2]])) - np.array([point[0], point[2]]), axis=1)
            mask_distance = distances <= threshold # only take out distance that <= 0.0001
            temp_wall_i = temp_wall_i[~mask_distance]
            
        wall_i = np.concatenate((cut_wall_i, temp_wall_i))
    return wall_i
        





In [20]:
def duc_tuong3(object_ins, objecter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.04):
    mask_door = objecter[:,0] <= (max_x + min_x) / 2
    door_true = object_ins[mask_door]
    for j in np.unique(door_true):# take out each door
        this_door = objecter[object_ins == j]
        this_masked_door = this_door[:,0] <= (max_x + min_x) / 2 - 0.05 # take out the point near the wall
        still_this_door = this_door[this_masked_door]
        
        if still_this_door.shape == (0,3):
            return wall_i
        # take min, max x,z to cut the wall
        min_y, max_y, min_z, max_z = np.min(still_this_door[:,1]), np.max(still_this_door[:,1]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
        hup1 = np.logical_and(wall_i[:,1] <= max_y, wall_i[:,1] >= min_y)
        hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
        hup_het = np.logical_and(hup1, hup2)
        
        temp_wall_i = wall_i[hup_het]
        cut_wall_i = wall_i[~hup_het]
        # 
        for point in still_this_door:
            distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,1], temp_wall_i[:,2]])) - np.array([point[1], point[2]]), axis=1)
            mask_distance = distances <= threshold # only take out distance that <= 0.0001
            temp_wall_i = temp_wall_i[~mask_distance]
        
        wall_i = np.concatenate((cut_wall_i, temp_wall_i))
    return wall_i
        





In [21]:
def duc_tuong4(object_ins, objecter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.04):
    mask_door = objecter[:,0] >= (max_x + min_x) / 2
    door_true = object_ins[mask_door]
    for j in np.unique(door_true):# take out each door
        this_door = objecter[object_ins == j]
        this_masked_door = this_door[:,0] >= (max_x + min_x) / 2 - 0.05 # take out the point near the wall
        still_this_door = this_door[this_masked_door]
        
        if still_this_door.shape == (0,3):
            return wall_i
        # take min, max x,z to cut the wall
        min_y, max_y, min_z, max_z = np.min(still_this_door[:,1]), np.max(still_this_door[:,1]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
        hup1 = np.logical_and(wall_i[:,1] <= max_y, wall_i[:,1] >= min_y)
        hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
        hup_het = np.logical_and(hup1, hup2)
        
        temp_wall_i = wall_i[hup_het]
        cut_wall_i = wall_i[~hup_het]
        # 
        for point in still_this_door:
            distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,1], temp_wall_i[:,2]])) - np.array([point[1], point[2]]), axis=1)
            mask_distance = distances <= threshold # only take out distance that <= 0.0001
            temp_wall_i = temp_wall_i[~mask_distance]
        
        wall_i = np.concatenate((cut_wall_i, temp_wall_i))
    return wall_i
        





In [22]:
def dbscaner(temp_wall_i,  cut_wall_i):
    db = DBSCAN(eps=0.2, min_samples = 10)
    labels = db.fit_predict(temp_wall_i)
    
    if len(np.unique(labels)) == 1:
        return cut_wall_i
    
    counter = []
    save_point = []
    
    for i in np.unique(labels):
        mask = labels ==i
        cluster_i = temp_wall_i[mask]
        count = len(cluster_i)
        counter.append(count)
        save_point.append(cluster_i)
        
    if len(counter) != 1:
        temp_wall_i = save_point[np.argmax(counter)]
        
    cut_wall_i = np.concatenate((cut_wall_i, temp_wall_i))

    return cut_wall_i

In [24]:
wall_dict = []
fix_point = 11000
for i in np.unique(wall_ins):
    mask = wall_ins == i
    wall_i = wall[mask]
    
    
    # print(sorted(wall_i[:,0] ))
    # print(sorted(wall_i[:,1] ))
    # print(sorted(wall_i[:,2] ))
    #GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(wall_i), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
    
    
    min_x, max_x, min_y, max_y, min_z, max_z = np.min(wall_i[:,0]), np.max(wall_i[:,0]), np.min(wall_i[:,1]), np.max(wall_i[:,1]), np.min(wall_i[:,2]), np.max(wall_i[:,2])
    print(min_x, max_x, min_y, max_y, min_z, max_z)
    
    if max_x - min_x >=2: # wall_x
        wall_i = wall_x(min_x, max_x, min_y, max_y, min_z, max_z, fix_point)
        if max_y <= 1: #north
            mask_door = door[:,1] <= (max_y + min_y) / 2
            door_true = door_ins[mask_door]
            #print(np.unique(door_true))
            for j in np.unique(door_true):# take out each door
                this_door = door[door_ins == j]
                this_masked_door = this_door[:,1] <= (max_y + min_y) / 2 + 0.05 # take out the point near the wall
                still_this_door = this_door[this_masked_door]
                # take min, max x,z to cut the wall
                min_x, max_x, min_z, max_z = np.min(still_this_door[:,0]), np.max(still_this_door[:,0]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
                hup1 = np.logical_and(wall_i[:,0] <= max_x, wall_i[:,0] >= min_x)
                hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
                hup_het = np.logical_and(hup1, hup2)
                
                temp_wall_i = wall_i[hup_het]
                cut_wall_i = wall_i[~hup_het]
                # 
                for point in still_this_door:
                    distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,0], temp_wall_i[:,2]])) - np.array([point[0], point[2]]), axis=1)
                    mask_distance = distances <= 0.04 # only take out distance that <= 0.0001
                    temp_wall_i = temp_wall_i[~mask_distance]
                
                wall_i = dbscaner(temp_wall_i, cut_wall_i)
                
            
                
            wall_i = duc_tuong1(window_ins, window, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            wall_i = duc_tuong1(board_ins, board, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            #wall_i = duc_tuong1(clutter_ins, clutter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.05)

        else:
            mask_door = door[:,1] >= (max_y + min_y) / 2
            door_true = door_ins[mask_door]
            #print(np.unique(door_true))
            for j in np.unique(door_true):# take out each door
                this_door = door[door_ins== j]
                this_masked_door = this_door[:,1] >= (max_y + min_y) / 2 + 0.05 # take out the point near the wall
                still_this_door = this_door[this_masked_door]
                # take min, max x,z to cut the wall
                min_x, max_x, min_z, max_z = np.min(still_this_door[:,0]), np.max(still_this_door[:,0]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
                hup1 = np.logical_and(wall_i[:,0] <= max_x, wall_i[:,0] >= min_x)
                hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
                hup_het = np.logical_and(hup1, hup2)
                
                temp_wall_i = wall_i[hup_het]
                cut_wall_i = wall_i[~hup_het]
                # 
                for point in still_this_door:
                    distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,0], temp_wall_i[:,2]])) - np.array([point[0], point[2]]), axis=1)
                    mask_distance = distances <= 0.04 # only take out distance that <= 0.0001
                    temp_wall_i = temp_wall_i[~mask_distance]
                
                #wall_i = np.concatenate((cut_wall_i, temp_wall_i))
                wall_i = dbscaner(temp_wall_i, cut_wall_i)
                
            wall_i = duc_tuong2(window_ins, window, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            wall_i = duc_tuong2(board_ins, board, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            #wall_i = duc_tuong2(clutter_ins, clutter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.05)
                
    elif max_y - min_y >=2:
        wall_i = wall_y(min_x, max_x, min_y, max_y, min_z, max_z, fix_point)
        
        if max_x <= 1: #east
            mask_door = door[:,0] <= (max_x + min_x) / 2
            door_true = door_ins[mask_door]
            for j in np.unique(door_true):# take out each door
                this_door = door[door_ins == j]
                this_masked_door = this_door[:,0] <= (max_x + min_x) / 2 - 0.05 # take out the point near the wall
                #print("this_masked_door", this_masked_door)
                still_this_door = this_door[this_masked_door]
                #print("still_this_door", still_this_door)
                # take min, max x,z to cut the wall
                min_y, max_y, min_z, max_z = np.min(still_this_door[:,1]), np.max(still_this_door[:,1]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
                hup1 = np.logical_and(wall_i[:,1] <= max_y, wall_i[:,1] >= min_y)
                hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
                hup_het = np.logical_and(hup1, hup2)
                
                temp_wall_i = wall_i[hup_het]
                cut_wall_i = wall_i[~hup_het]
                # 
                for point in still_this_door:
                    distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,1], temp_wall_i[:,2]])) - np.array([point[1], point[2]]), axis=1)
                    mask_distance = distances <= 0.04 # only take out distance that <= 0.0001
                    temp_wall_i = temp_wall_i[~mask_distance]
                    
                    
                
                wall_i = dbscaner(temp_wall_i, cut_wall_i)
                
                
                
            wall_i = duc_tuong3(window_ins, window, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            wall_i = duc_tuong3(board_ins, board, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            #wall_i = duc_tuong3(clutter_ins, clutter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.05)
        else: # west
            mask_door = door[:,0] >= (max_x + min_x) / 2
            door_true = door_ins[mask_door]
            #print(np.unique(door_true))
            for j in np.unique(door_true):# take out each door
                this_door = door[door_ins == j]
                this_masked_door = this_door[:,0] >= (max_x + min_x) / 2 - 0.05 # take out the point near the wall
                #print("this_masked_door", this_masked_door)
                still_this_door = this_door[this_masked_door]
                #print("still_this_door", still_this_door)
                # take min, max x,z to cut the wall
                min_y, max_y, min_z, max_z = np.min(still_this_door[:,1]), np.max(still_this_door[:,1]), np.min(still_this_door[:,2]), np.max(still_this_door[:,2])
                hup1 = np.logical_and(wall_i[:,1] <= max_y, wall_i[:,1] >= min_y)
                hup2 = np.logical_and(wall_i[:,2] <= max_z, wall_i[:,2] >= min_z)
                hup_het = np.logical_and(hup1, hup2)
                
                temp_wall_i = wall_i[hup_het]
                #print("temp_wall_i", temp_wall_i)
                cut_wall_i = wall_i[~hup_het]
                # 
                for point in still_this_door:
                    distances       = np.linalg.norm(np.transpose(np.array([temp_wall_i[:,1], temp_wall_i[:,2]])) - np.array([point[1], point[2]]), axis=1)
                    #print("distances", distances)
                    mask_distance = distances <= 0.04 # only take out distance that <= 0.0001
                    #print("mask_distance", mask_distance)
                    temp_wall_i = temp_wall_i[~mask_distance]
                
                wall_i = dbscaner(temp_wall_i, cut_wall_i)
                
            wall_i = duc_tuong4(window_ins, window, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            wall_i = duc_tuong4(board_ins, board, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.08)
            
            #wall_i = duc_tuong4(clutter_ins, clutter, wall_i, min_x, max_x, min_y, max_y, min_z, max_z, threshold = 0.05)
    wall_dict.append(wall_i)
    #GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(wall_i), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])

wall_complete = np.concatenate(wall_dict)
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(wall), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(wall_complete), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
    

6.048 6.108 0.509 5.829 0.044 3.064
0.318 5.208 0.009 0.109 0.054 3.064
0.638 5.268 6.329 6.429 0.044 3.064
0.199 0.623 0.072 6.027 0.052 3.043


In [47]:
new_floor = floor1(np.min(floor[:,0]), np.max(floor[:,0]), np.min(floor[:,1]), np.max(floor[:,1]), np.min(floor[:,2]), np.max(floor[:,2]), 8290)

GeometricTools.drawPointCloudsColorsClasses(array = torch.concatenate((torch.tensor(floor), (torch.tensor(new_floor) + torch.tensor([1,0,0])*8))), colors = torch.tensor([0,1]), origins = [[0,0,0]])

In [48]:
new_ceiling = floor1(np.min(ceiling[:,0]), np.max(ceiling[:,0]), np.min(ceiling[:,1]), np.max(ceiling[:,1]), np.min(ceiling[:,2]), np.max(ceiling[:,2]), 8290)

GeometricTools.drawPointCloudsColorsClasses(array = torch.concatenate((torch.tensor(ceiling), (torch.tensor(new_ceiling) + torch.tensor([1,0,0])*8))), colors = torch.tensor([0,1]), origins = [[0,0,0]])

In [55]:
not_wall = np.concatenate((door, window, board, clutter))
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(not_wall), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])

In [58]:
wall_column = np.concatenate((wall_complete, room_not_wall))
room_new_floor = np.concatenate((wall_complete, room_not_cc, new_floor,new_ceiling))
GeometricTools.drawPointCloudsColorsClasses(array = torch.concatenate((torch.tensor(room_with_wall),torch.tensor(room_new_floor) + torch.tensor([9,0,0]))) , colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(room_with_wall), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])


In [14]:
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(door), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(window), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])

GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(board), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])
GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(clutter), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])

In [ ]:
print(np.unique(clutter_ins))
print(np.unique(door_ins))
print(np.unique(window_ins))
print(np.unique(board_ins))

[ 1  3  8 10 11 12 16 18 19 22 25 26 27 29 30 32 33 35 37 38 40 41 43 47
 48 49 51 55 57 59 60 61 62 63 64 66 68 69 72 73 78 79 81 83 84 86 90]
[17 53]
[56 74]
[9]


In [16]:
def split_wall(wall_complete):
    min_x, max_x, min_y, max_y, min_z, max_z = np.min(wall_complete[:,0]), np.max(wall_complete[:,0]), np.min(wall_complete[:,1]), np.max(wall_complete[:,1]), np.min(wall_complete[:,2]), np.max(wall_complete[:,2])

    wall_split = 0
    ins_split = 0
    for i in range(4):
        if i ==0:
            mask = wall_complete[:,1] <=  min_y+ 0.07
            wall_i = wall_complete[mask]
            wall_i_ins = np.ones((wall_i.shape[0],))
            wall_split = wall_i
            ins_split = wall_i_ins
        elif i ==1:
            mask = wall_complete[:,0] <= min_x  + 0.07
            wall_i = wall_complete[mask]
            wall_i_ins = np.ones((wall_i.shape[0],)) * 2
            
            wall_split = np.concatenate((wall_split, wall_i))
            ins_split = np.concatenate((ins_split, wall_i_ins))
        if i ==2:
            mask = wall_complete[:,1] >= max_y - 0.07
            wall_i = wall_complete[mask]
            wall_i_ins = np.ones((wall_i.shape[0],)) * 3
            
            wall_split = np.concatenate((wall_split, wall_i))
            ins_split = np.concatenate((ins_split, wall_i_ins))
        if i ==3:
            mask = wall_complete[:,0] >= max_x - 0.07
            wall_i = wall_complete[mask]
            wall_i_ins = np.ones((wall_i.shape[0],)) * 4
            
            wall_split = np.concatenate((wall_split, wall_i))
            ins_split = np.concatenate((ins_split, wall_i_ins))
            
        
    #GeometricTools.drawPointCloudsColorsClasses(array = torch.tensor(wall_split), colors = torch.tensor([1,1,1]), origins = [[0,0,0]])



In [ ]:
def move_from_wall(objecter, min_X, max_X, min_Y, max_Y, min_Z):
    min_x, max_x, min_y, max_y, min_z, max_z = np.min(objecter[:,0]), np.max(objecter[:,0]), np.min(objecter[:,1]), np.max(objecter[:,1]), np.min(objecter[:,2]), np.max(objecter[:,2])
    
    #for i in range(4):
    if min_z < min_Z:
            objecter[:,2] += min_Z - min_z
    if min_x < min_X:
        objecter[:,0] += min_X - min_x
    if min_x < min_Y:
        objecter[:,1] += min_Y - min_y
    if max_x > max_X:
        objecter[:,0] -= max_x - max_X
    if max_y > max_Y:
        objecter[:,1] -= max_Y - max_y
        
    return objecter